## PSO optimizer tutorial.
### Burgers equation.
$$\frac{\partial u}{\partial t}+ u\frac{\partial u}{\partial x}=\mu\frac{\partial^2 u}{\partial x^2} $$
$$\mu=0.02/\pi$$
$$x\in[-1,1]$$
$$t\in[0,1]$$

*Initial and boundary conditions*
$$u(x, t=0)=-sin(\pi*x)$$
$$u(x=-1, t)=0$$
$$u(x=1, t)=0$$

import libraries and Solver modules.

In [9]:
import torch
import numpy as np
from scipy.integrate import quad
import sys
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('tutorials'), '..')))

from tedeous.data import Domain, Conditions, Equation
from tedeous.model import Model
from tedeous.callbacks import early_stopping, plot
from tedeous.optimizers.optimizer import Optimizer
from tedeous.device import solver_device, check_device
from tedeous.models import mat_model

Building grid, boundary conditions, equation.

In [10]:
solver_device('cuda')

mode = 'autograd'

mu = 0.02 / np.pi

##Domain class for doamin initialization
domain = Domain()
domain.variable('x', [-1, 1], 20)
domain.variable('t', [0, 1], 20)

boundaries = Conditions()

##initial cond
x = domain.variable_dict['x']
boundaries.dirichlet({'x': [-1, 1], 't': 0}, value=-torch.sin(np.pi*x))

##boundary cond
boundaries.dirichlet({'x': -1, 't': [0, 1]}, value=0)

##boundary cond
boundaries.dirichlet({'x': 1, 't': [0, 1]}, value=0)

equation = Equation()

## equation part
burgers_eq = {
    'du/dt**1':
        {
            'coeff': 1.,
            'du/dt': [1],
            'pow': 1,
            'var': 0
        },
    '+u*du/dx':
        {
            'coeff': 1,
            'u*du/dx': [[None], [0]],
            'pow': [1, 1],
            'var': [0, 0]
        },
    '-mu*d2u/dx2':
        {
            'coeff': -mu,
            'd2u/dx2': [0, 0],
            'pow': 1,
            'var': 0
        }
}

equation.add(burgers_eq)


CUDA is available and used.


exact solution

In [11]:
def exact(grid):
    mu = 0.02 / np.pi

    def f(y):
        return np.exp(-np.cos(np.pi * y) / (2 * np.pi * mu))

    def integrand1(m, x, t):
        return np.sin(np.pi * (x - m)) * f(x - m) * np.exp(-m ** 2 / (4 * mu * t))

    def integrand2(m, x, t):
        return f(x - m) * np.exp(-m ** 2 / (4 * mu * t))

    def u(x, t):
        if t == 0:
            return -np.sin(np.pi * x)
        else:
            return -quad(integrand1, -np.inf, np.inf, args=(x, t))[0] / quad(integrand2, -np.inf, np.inf, args=(x, t))[
                0]

    solution = []
    for point in grid:
        solution.append(u(point[0].item(), point[1].item()))

    return torch.tensor(solution)

When grid, equation, boundary conditions  exist, we should call preprocessing class Equation with method set_strategy and initialize model.

In [12]:
## model part
if mode in ('NN', 'autograd'):
    net = torch.nn.Sequential(
        torch.nn.Linear(2, 10),
        torch.nn.Tanh(),
        torch.nn.Linear(10, 10),
        torch.nn.Tanh(),
        torch.nn.Linear(10, 10),
        torch.nn.Tanh(),
        torch.nn.Linear(10, 1)
    )
else:
    net = mat_model(domain, equation)

After that, we can initialize optimizer, it may be one of torch optimizers or custom form *tedeous.optimizers* module.
If you want to run optimizatoin process with default settings,
you will be able to set it as string ("Adam", "SGD", "LBFGS", "PSO")

*Here is main aspects of particle swarm optimizer realization*
1. For optimization, the swarm *pop_size* is built based on initial model weights with adding some *variance* (influence on search space)
2. Each individual in this swarm represents a candidate solution. At each iteration, the particles in the swarm exchange
information and use it to update their positions.
3.  Particle $\theta^t$ at iteration $t$ is changed by three factors: its own velocity inertia $\beta \upsilon^t$
, its best-known position $p_{best}$ in the search-space, as well as the
entire swarm’s best-known position $g_{best}$:
$$\upsilon^{t+1} = \beta*\upsilon^{t} + (1-\beta)*(c1*r1(p_{best} − \theta^t) + c2*r2(g_{best} − \theta^t))$$
where *c1* and *c2* are the cognitive and social coefficients, respectively, referred to jointly as the behavioral
coefficients, and *r1* and *r2* are uniformly distributed random numbers in range (-*variance*, *variance*). Then the particle position is updated as:
$$\theta^{t+1} = \theta^t + \upsilon^{t+1}$$
4. PSO can be combined with gradient descent to train neural networks:
$$v^{t+1} = \beta*\upsilon^{t} + (1-\beta)*(c1*r1(p_{best} − \theta^t) + c2*r2(g_{best} − \theta^t)) − \alpha*\nabla Loss(\theta^t)$$
where $\alpha$ is *lr*.

Based on formulaes above, here is matching formulaes coef-nts with *PSO* algorithm parameters:
1. pop_size (int, optional): The swarm. Defaults to 30.
2. b (float, optional): Inertia of the particles. Defaults to 0.9.
3. c1 (float, optional): The *p-best* coeficient. Defaults to 0.08.
4. c2 (float, optional): The *g-best* coeficient. Defaults to 0.5.
5. c_decrease (bool, optional): Flag for update_pso_params method. Defautls to False.
6. variance (float, optional): Variance parameter for swarm creation
based on init model, ifluence on r1 and r2 coeff-nts. Defaults to 1.
7. lr (float, optional): Learning rate for gradient descent. Defaults to 1e-3.
    If 0, there will be only PSO optimization without gradients.
8. epsilon (float, optional): some add to gradient descent like in Adam optimizer. Defaults to 1e-8.

After preliminaries, to sart solving the equation, we should call Solver class with method solve:

In [13]:
img_dir=os.path.join(os.path.dirname('tutorials'), 'Burg_eq_img')

model = Model(net, domain, equation, boundaries)

model.compile(mode, lambda_operator=1, lambda_bound=10)

cb_es = early_stopping.EarlyStopping(eps=1e-7,
                                     loss_window=100,
                                     no_improvement_patience=1000,
                                     patience=3,
                                     randomize_parameter=1e-5,
                                     info_string_every=1000)

cb_plots = plot.Plots(save_every=1000, print_every=None, img_dir=img_dir)


In [14]:
optimizer = Optimizer('Adam', {'lr': 1e-3})

model.train(optimizer, 10000, save_model=False, callbacks=[cb_es, cb_plots])

grid = domain.build(mode)

u_exact = exact(grid).to('cuda')

u_exact = check_device(u_exact).reshape(-1)

u_pred = check_device(net(grid)).reshape(-1)

error_rmse = torch.sqrt(torch.sum((u_exact - u_pred)**2)) / torch.sqrt(torch.sum(u_exact**2))

print('RMSE_grad= ', error_rmse.item())

[2024-05-11 00:09:18.373698] initial (min) loss is 4.058678150177002
[2024-05-11 00:09:37.501029] Step = 1000 loss = 0.265380 normalized loss line= -0.000479x+1.050024. There was 0 stop dings already.
[2024-05-11 00:09:56.623818] Step = 2000 loss = 0.183178 normalized loss line= -0.000267x+1.028287. There was 0 stop dings already.
[2024-05-11 00:10:13.856850] Step = 3000 loss = 0.137374 normalized loss line= -0.000589x+1.064996. There was 0 stop dings already.
[2024-05-11 00:10:31.337792] Step = 4000 loss = 0.029479 normalized loss line= -0.001481x+1.153410. There was 0 stop dings already.
[2024-05-11 00:10:48.801780] Step = 5000 loss = 0.009205 normalized loss line= 0.000204x+1.063518. There was 0 stop dings already.
[2024-05-11 00:11:06.691818] Step = 6000 loss = 0.005344 normalized loss line= -0.000862x+1.123907. There was 0 stop dings already.
[2024-05-11 00:11:24.454755] Step = 7000 loss = 0.004046 normalized loss line= -0.000225x+1.023839. There was 0 stop dings already.
[2024-05

C:\Users\Рустам\AppData\Local\Temp\ipykernel_28972\2125334694.py:17: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return -quad(integrand1, -np.inf, np.inf, args=(x, t))[0] / quad(integrand2, -np.inf, np.inf, args=(x, t))[
C:\Users\Рустам\AppData\Local\Temp\ipykernel_28972\2125334694.py:17: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  return -quad(integrand1, -np.inf, np.inf, args=(x, t))[0] / quad(integrand2, -np.inf, np.inf, args=(x, t))[


RMSE_grad=  0.0375038145958498


for trained model we want to start PSO.

In [15]:
optimizer = Optimizer('PSO', {'pop_size': 10,
                              'b': 0.5,
                              'c2': 0.05,
                              'variance': 5e-3,
                              'c_decrease': True,
                              'lr': 5e-3})

model.train(optimizer, 3000, info_string_every=100, save_model=False, callbacks=[cb_es, cb_plots])

u_pred = check_device(net(grid)).reshape(-1)

error_rmse = torch.sqrt(torch.sum((u_exact - u_pred)**2)) / torch.sqrt(torch.sum(u_exact**2))

print('RMSE_pso= ', error_rmse.item())

[2024-05-11 00:12:26.328809] initial (min) loss is 0.007288636174052954
---------------------------------------------------
tensor([0.0096, 0.0078, 0.0167, 0.0176, 0.0119, 0.0064, 0.0077, 0.0339, 0.0096,
        0.0157], device='cuda:0')
tensor([3.8948, 3.0308, 3.5419, 0.5601, 3.0309, 1.3399, 2.8512, 2.5814, 2.9880,
        2.9912], device='cuda:0')
tensor(0.0064, device='cuda:0')
tensor([0.0008], device='cuda:0')
---------------------------------------------------
tensor([0.0096, 0.0078, 0.0167, 0.0176, 0.0119, 0.0064, 0.0077, 0.0339, 0.0096,
        0.0157], device='cuda:0')
tensor([0.6091, 0.1961, 0.9056, 2.3373, 0.1865, 2.4077, 0.1736, 0.0821, 0.2306,
        0.1504], device='cuda:0')
tensor(0.0064, device='cuda:0')
tensor([0.0053], device='cuda:0')
---------------------------------------------------
tensor([0.0096, 0.0078, 0.0167, 0.0176, 0.0119, 0.0064, 0.0077, 0.0339, 0.0096,
        0.0157], device='cuda:0')
tensor([1.5609, 0.5878, 1.2190, 0.7047, 0.5723, 0.0776, 0.6422, 1.0036